# ℹ️ This Notebook was created for educational proposal

## Installing libraries

In [ ]:
!pip install pybase64

In [ ]:
!pip install python-magic

## Importing libraries

In [ ]:
import bs4
import pandas as pd
import pybase64
import magic
import mimetypes
import os
import urllib
import hashlib
from tqdm import tqdm

## Reading index.html

In [ ]:
html = None
with open('index.html','r') as file:
  html = file.read()

In [ ]:
soup = bs4.BeautifulSoup(html)

## Getting images tags

In [ ]:
html_div = soup.find('div', {'id':'islrg'})

In [ ]:
src_list = []
for tag in html_div.find_all('img'):
  try:
    src_list.append(tag['src'])
  except:
    pass

## Saving data in a pandas DataFrame

In [ ]:
df = pd.DataFrame({'src':src_list})

## Removing duplicated and icons images

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df[df['src'].str.contains('favicon') == False]

## Function save images

In [ ]:
def base64_to_image(base64:str,output_path:str):
  base64 = base64.split('base64')
  format = None
  if 'jpeg' in base64[0]:
    format = 'jpg'
  elif 'png' in base64[0]:
    format = 'png'
  elif 'gif' in base64[0]:
    format = 'gif'
  else:
    raise Exception('Unknown image extension')

  base64 = base64[1][1:]

  decoded_b64 = pybase64.b64decode((base64))
  file_name = f'{hashlib.md5(base64.encode()).hexdigest()}.{format}'
  file_path = os.path.join(output_path,file_name)
  img_file = open(file_path, 'wb')
  img_file.write(decoded_b64)
  img_file.close()

In [ ]:
def url_to_image(url:str,output_path:str):
  mime = magic.Magic(mime=True)
  output = hashlib.md5(url.encode()).hexdigest()
  file_path = os.path.join(output_path,output)
  urllib.request.urlretrieve(url,file_path)
  mimes = mime.from_file(file_path)
  ext = mimetypes.guess_all_extensions(mimes)[0]
  os.rename(file_path, file_path+ext)

## Save all the images in 'dataset/images' folder

In [ ]:
print('Saving the images. Please wait...')
output_path = 'dataset/images'
if os.path.exists(output_path) == False:
  os.makedirs(output_path)
for row in tqdm(df.iterrows()):
  src = row[1]['src']
  if src[0:4] == 'data':
    base64_to_image(src,output_path)
  elif src[0:4] == 'http':
    url_to_image(src,output_path)
  else:
    raise Exception('Unknown source')
print()
print('Images saved successfully')
print('Total:',df.shape[0],'images')

Saving the images. Please wait...


434it [00:16, 26.96it/s]

Images saved successfully
Total: 434 images


## Compress and save images in './images.tar.gz'


In [ ]:
!tar czf images.tar.gz dataset/images/